In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import enum
import re
import nltk 
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [2]:
#Veri setinin yüklenmesi ve örnek veri
dataset = pd.read_csv("data.csv",delimiter=";",header=None,names=["Review","Rating"])
print("Verisetinde {} adet cümle mevcut.".format(len(dataset)))
dataset.head()

Verisetinde 50000 adet cümle mevcut.


,Review,Rating
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [3]:
#Veri ön işleme
#Ön işleme öncesi örnek cümle
print(dataset['Review'].values[0],"\n\n")

#stopwords ön hazırlık
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('english')
print(stop_word_list)

def token(values):
    words = nltk.tokenize.word_tokenize(values)
    filtered_words = [word for word in words if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc

#stemming ön hazırlık
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

#Stemmer nesnesi oluşturulması
porter = PorterStemmer() 
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

#lemmatization ön hazırlık
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

wordnet_lemmatizer = WordNetLemmatizer()
# Lemmatize with POS Tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmaSentence(sentence):
    token_words=word_tokenize(sentence)
    lemma_sentence=[]
    for word in token_words:
        lemma_sentence.append(wordnet_lemmatizer.lemmatize(word,pos='v'))
        lemma_sentence.append(" ")
    return "".join(lemma_sentence)

#büyük harflerin küçük harfe çevrilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: x.lower())

# Özel karakterlerin(noktalama işareti vs) çıkartılması
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\W", " ", x))

# tek karakterlerin boşluk ile değiştirilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\s+[a-zA-Z]\s+", " ", x))

# en baştan tek kalan karakterlerin çıkartılması
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\^[a-zA-Z]\s+", " ", x))

# Birden fazla boşluğun tek boşlukla değiştirilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\s+", " ", x))

# b öneklerinin silinmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"^b\s+", " ", x))

#fazladan boşlukların temizlenmesi
dataset['Review'] = dataset['Review'].apply(lambda x: x.strip())

#stopwordlerin temizlenmesi
dataset['Review'] = dataset['Review'].apply(lambda x: token(x))

#Lemmatization işlemi
dataset['Review'] = dataset['Review'].apply(lambda x: lemmaSentence(x))

#stemming işlemi
dataset['Review'] = dataset['Review'].apply(lambda x: stemSentence(x))
                              
data = dataset['Review'].values.tolist()
target = dataset['Rating'].values.tolist()

#Ön işleme sonrası aynı cümle
print(data[0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [4]:
#Cümlelerin eğitim ve test olarak ayrılması %90 Eğitim %10 test
ratio = int(len(data) * .90)
x_train, y_train = data[:ratio], target[:ratio]
y_train = np.array(y_train)
x_test, y_test   = data[ratio:], target[ratio:]
y_test = np.array(y_test)

print("{} adet cümle eğitim için kullanılacak.".format(len(x_train)))
print("{} adet cümle test için kullanılacak.".format(len(x_test)))

45000 adet cümle eğitim için kullanılacak.
5000 adet cümle test için kullanılacak.


In [5]:
#Cümlelerin içinde geçen kelimelerden 10000 kelimelik bir sözlük oluşturuluyor.
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
#tokenizer.word_index

#Cümleler sayılara dönüştürülüyor
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

#Cümlelerin önceki ve sonraki hallerinin görüntülenmesi
IDX = 0
print("Öncesi: {}".format(x_train[IDX]))
print("Sonrası: {}".format(np.array(x_train_tokens[IDX])))

Öncesi: one review mention watch 1 oz episod hook right exactli happen br br first thing strike oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam give oswald maximum secur state penitentari focu mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death star dodgi deal shadi agreement never far away br br would say main appeal show due fact go show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw strike nasti surreal say readi watch develop tast oz get accustom high level graphic violenc violenc injustic crook guard sell nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get tou

In [6]:
#RNN'e girdileri vermeden önce tamamının aynı boyutta olması gerekli. Bu sebeple aşağıdaki matematiksel işlemleri yapıyoruz.
total_sentences = x_train_tokens + x_test_tokens
num_tokens = np.array([len(tokens) for tokens in total_sentences])
#print(np.mean(num_tokens))
#print(np.std(num_tokens))
#print(np.max(num_tokens))
#print(np.min(num_tokens))

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) # np.std = standart sapma
max_tokens = int(max_tokens)
print(max_tokens)
#Verinin ne kadarını bu kapsama aldığımızın ölçülmesi
print("%", round(np.sum(num_tokens < max_tokens) / len(num_tokens) * 100, 2))

292
% 94.48


In [7]:
#Padding işlemi. Bulunan uzunluk değerine göre cümlelerin yeniden düzenlenmesi. Kısa olanların başına sıfır eklenmesi.
#Uzun olanlardan baştan silme yapılması
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad  = pad_sequences(x_test_tokens,  maxlen=max_tokens)

In [8]:
#RNN oluşturma
#ardışık bir model
model = Sequential()
    
#her kelimeye karşılık gelen 50 uzunluğunda bir vektör oluşturulur. (Embedding matrisi)
embedding_size = 50
    
#matris kelime sayısı ve embedding büyüklüğünde olacak, yani 10bine 50 uzunluğunda 
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))
#LSTM layerlerinin eklenmesi
# 16 nöronlu LSTM (16 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(GRU(units=16, return_sequences=True))
# 8 nöronlu LSTM (8 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(GRU(units=8, return_sequences=True))
# 4 nöronlu LSTM (4 outputlu , return_sequences=False yani default değer, tek bir output verecek)
model.add(GRU(units=4))
# Tek bir nörondan oluşan output layer'ı
model.add(Dense(1, activation='sigmoid'))

#modelin derlenmesi 
#iki sınıf olduğu için loss fonksiyonu olarak binary_crossentropy 
#modelin başarısını görmek için accuracy metrics
#optimizasyon algoritması
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [9]:
#Modelin özeti
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 292, 50)           500000    
_________________________________________________________________
gru (GRU)                    (None, 292, 16)           3264      
_________________________________________________________________
gru_1 (GRU)                  (None, 292, 8)            624       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [10]:
#model eğitimi, bir defa eğitimden geçmesi -> epoch , batch_size -> 256'şar 256'şar beslenecek.
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

Epoch 1/5
176/176 [==============================] - 46s 232ms/step - loss: 0.5933 - accuracy: 0.6883
Epoch 2/5
176/176 [==============================] - 40s 230ms/step - loss: 0.2893 - accuracy: 0.8967
Epoch 3/5
176/176 [==============================] - 42s 236ms/step - loss: 0.2146 - accuracy: 0.9289
Epoch 4/5
176/176 [==============================] - 41s 234ms/step - loss: 0.1797 - accuracy: 0.9419
Epoch 5/5
176/176 [==============================] - 41s 233ms/step - loss: 0.1566 - accuracy: 0.9522


In [11]:
#Evaluate fonksiyonu yalnızca accuracy ve loss değerini döndürür
result = model.evaluate(x_test_pad, y_test)

num_true_sentence = int(len(x_test) * result[1])
print("Test verisindeki {} adet cümleden {} tanesi doğru bilindi.".format(len(x_test), num_true_sentence))

157/157 [==============================] - 5s 27ms/step - loss: 0.3055 - accuracy: 0.8854
Test verisindeki 5000 adet cümleden 4426 tanesi doğru bilindi.


In [12]:
#tek tek cümlelerin sonuçlarını görmek için predict metodu kullanılması
y_pred = model.predict(x_test_pad)

#Her cümle için çıktı 0 ile 1 arasındadır. 0 olumsuz 1 olumlu anlamındadır. 
#0.5 üzerini olumlu altını olumsuz olarak işaretleyelim.
y_pred = np.array([1 if p>0.5 else 0 for p in y_pred])

#Bir örnek üzerinde inceleyelim.
IDX = 0
sentence = x_test[IDX]
real_rate = y_test[IDX]
predicted_rate = y_pred[IDX]

print("Cümle: {} \nAsıl Etiket: {} \nÜretilen Etiket: {}".format(sentence, real_rate, predicted_rate))

Cümle: enjoy film sceneri corfu greek ador countri like flatter director point view base true stori year greec struggl stand two feet war nazi hardship italian soldier greek girl fall love time hard lot sacrific make nichola cage look great uniform give passion account unfulfil begin love ador christian bale play mandra heroin husband look good greek person match one greek patriot true fighter one movi would like buy keep collect ever  
Asıl Etiket: 1 
Üretilen Etiket: 1


In [13]:
#Accuracy değeri hariç precision,recall ve f-measure değerlerine bakalım.
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[2193  337]
 [ 236 2234]]
              precision    recall  f1-score   support

           0       0.90      0.87      0.88      2530
           1       0.87      0.90      0.89      2470

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

